In [170]:
# Import setup and pandas_style_helpers
from setup.setup import *
from matplotlib.colors import Colormap

In [2]:
# Load in data
cdlDF = load_and_clean_cdl_data()
cdlDF


c:\Users\David Harler Jr\OneDrive\Desktop\dataClass\06-cod-analysis\2024.04.10 CDL PrizePicks Betting\cdl_prizepicks_betting\cdl-betting-app-v02\setup\setup.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cdlDF = sqlio.read_sql_query("SELECT * FROM cdl_data", conn)


,match_id,match_date,match_day,player,team,map_num,map_name,gamemode,kills,deaths,...,series_result,map_wl,team_abbr,team_icon,opp,opp_abbr,opp_score,total_score,score_diff,dummy_x
0,27193,2023-12-08,Friday,aBeZy,Atlanta FaZe,1,Karachi,Hardpoint,24,24,...,1,W,ATL,FaZe,Boston Breach,BOS,238,488,12,0
1,27193,2023-12-08,Friday,Cellium,Atlanta FaZe,1,Karachi,Hardpoint,21,19,...,1,W,ATL,FaZe,Boston Breach,BOS,238,488,12,0
2,27193,2023-12-08,Friday,Drazah,Atlanta FaZe,1,Karachi,Hardpoint,25,22,...,1,W,ATL,FaZe,Boston Breach,BOS,238,488,12,0
3,27193,2023-12-08,Friday,Simp,Atlanta FaZe,1,Karachi,Hardpoint,33,23,...,1,W,ATL,FaZe,Boston Breach,BOS,238,488,12,0
4,27193,2023-12-08,Friday,Capsidal,Boston Breach,1,Karachi,Hardpoint,24,26,...,0,L,BOS,Breach,Atlanta FaZe,ATL,250,488,-12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4011,27284,2024-04-14,Sunday,Simp,Atlanta FaZe,4,Rio,Hardpoint,26,28,...,1,W,ATL,FaZe,OpTic Texas,TX,241,491,9,0
4012,27284,2024-04-14,Sunday,Dashy,OpTic Texas,4,Rio,Hardpoint,25,23,...,0,L,TX,OpTic,Atlanta FaZe,ATL,250,491,-9,0
4013,27284,2024-04-14,Sunday,Kenny,OpTic Texas,4,Rio,Hardpoint,27,26,...,0,L,TX,OpTic,Atlanta FaZe,ATL,250,491,-9,0
4014,27284,2024-04-14,Sunday,Pred,OpTic Texas,4,Rio,Hardpoint,21,25,...,0,L,TX,OpTic,Atlanta FaZe,ATL,250,491,-9,0


In [3]:
# Build team summaries
team_summaries_DF = build_team_summaries(cdlDF)
team_summaries_DF.head(20)

,team,gamemode,map_name,wins,losses,win_percentage
0,Atlanta FaZe,Hardpoint,6 Star,0,1,0.00
1,Atlanta FaZe,Hardpoint,Invasion,6,3,0.67
2,Atlanta FaZe,Hardpoint,Karachi,7,2,0.78
3,Atlanta FaZe,Hardpoint,Rio,5,0,1.00
5,Atlanta FaZe,Hardpoint,Sub Base,1,5,0.17
7,Atlanta FaZe,Hardpoint,Vista,1,0,1.00
8,Atlanta FaZe,Search & Destroy,Highrise,5,3,0.62
9,Atlanta FaZe,Search & Destroy,Invasion,9,3,0.75
10,Atlanta FaZe,Search & Destroy,Karachi,3,1,0.75
11,Atlanta FaZe,Search & Destroy,Rio,3,0,1.00


In [7]:
team_summaries_DF = \
    team_summaries_DF[
        ~((team_summaries_DF['gamemode'] == 'Hardpoint') & (team_summaries_DF['map_name'] == 'Invasion'))
    ]

team_summaries_DF

,team,gamemode,map_name,wins,losses,win_percentage
0,Atlanta FaZe,Hardpoint,6 Star,0,1,0.00
2,Atlanta FaZe,Hardpoint,Karachi,7,2,0.78
3,Atlanta FaZe,Hardpoint,Rio,5,0,1.00
5,Atlanta FaZe,Hardpoint,Sub Base,1,5,0.17
7,Atlanta FaZe,Hardpoint,Vista,1,0,1.00
...,...,...,...,...,...,...
222,Seattle Surge,Search & Destroy,Overall,17,8,0.68
223,Seattle Surge,Control,Overall,3,17,0.15
224,Toronto Ultra,Hardpoint,Overall,28,6,0.82
225,Toronto Ultra,Search & Destroy,Overall,15,11,0.58


In [8]:
# Check shape 
team_summaries_DF.shape

(170, 6)

In [181]:
# Function to build team summary
def team_summaries_fn(team_summaries_input: pd.DataFrame, team_x: str, team_y: str):
    summary_df = \
        pd.merge(
            team_summaries_input[team_summaries_input["team"] == team_x], 
            team_summaries_input[team_summaries_input["team"] == team_y], 
            how = "outer", 
            on = ["gamemode", "map_name"]
            ) \
            .drop(columns = ["team_x", "team_y"], inplace = False, axis = 1) \
            .sort_values(by = ["gamemode", "map_name"])
    
    # Fill NA's
    numerical_cols = summary_df.select_dtypes(include=['number']).columns
    summary_df[numerical_cols] = summary_df[numerical_cols].fillna(0)

    # Correct datatypes
    summary_df['wins_x'] = summary_df['wins_x'].astype('int64')
    summary_df['losses_x'] = summary_df['losses_x'].astype('int64')
    summary_df['wins_y'] = summary_df['wins_y'].astype('int64')
    summary_df['losses_y'] = summary_df['losses_y'].astype('int64')
    summary_df['win_percentage_x'] = summary_df['win_percentage_x'].astype('float64')
    summary_df['win_percentage_y'] = summary_df['win_percentage_y'].astype('float64')

    # Rename gamemode and map_name columns
    summary_df = summary_df.rename(columns = {"gamemode": "Gamemode", "map_name": "Map"})

    # Add MultiIndex to Rows
    summary_df = summary_df.set_index(["Gamemode", "Map"])

    # Add MultiIndex to Columns
    columns = pd.MultiIndex.from_tuples([
        (team_x, 'Wins'), (team_x, 'Losses'), (team_x, 'Win %'),
        (team_y, 'Wins'), (team_y, 'Losses'), (team_y, 'Win %')
    ])
    summary_df.columns = columns
    
    # Style dataframe
    summary_df_styler = summary_df.style \
        .format(precision = 2) \
        .background_gradient(vmin = 0, vmax = 1, axis = 0, cmap = Colormap("bwr").reversed, 
                             subset = [(team_x, 'Win %'), (team_y, 'Win %')])

    return summary_df_styler


In [182]:
team_summary_output = team_summaries_fn(team_summaries_DF, "New York Subliners", "Los Angeles Thieves")
team_summary_output

TypeError: get_cmap expects None or an instance of a str or Colormap . you passed <bound method Colormap.reversed of <matplotlib.colors.Colormap object at 0x00000256F422A2D0>> of type <class 'method'>